In [ ]:
def obtener_email(row, df_bancos, df_entidades_financieras):
    if row['IDs_entidades'] == 0 and row['IDs_entidades_bancos'] != 0:
        email_banco = df_bancos[df_bancos['Código'] == row['IDs_entidades_bancos']]['email']
        return email_banco.values[0] if not email_banco.empty else None
    elif row['IDs_entidades_bancos'] == 0 and row['IDs_entidades'] != 0:
        email_entidad = df_entidades_financieras[df_entidades_financieras['ID PSP'] == row['IDs_entidades']]['CONTACTO']
        return email_entidad.values[0] if not email_entidad.empty else None
    else:
        return None

df_resultado['email'] = df_resultado.apply(obtener_email, axis=1, args=(df_bancos, df_entidades_financieras))

In [ ]:
import pandas as pd
import pyodbc


df_cuentas = pd.read_excel('registro_tickets.xlsx', dtype=str)


valores_a_buscar = df_cuentas.apply(lambda row: row['Cuenta a bloquear'] if pd.notnull(row['Cuenta a bloquear']) else row['CUIT a bloquear'], axis=1)
valores_a_buscar_lista = valores_a_buscar.to_list()


df_cuentas['cuentas_reportar'] = None

conexion_ccvu = pyodbc.connect('Driver={SQL Server}; Server=tcp:10.152.0.42,1433; Database=ccvu; Trusted_connection=yes')
cursor_ccvu = conexion_ccvu.cursor()


nuevas_filas = []


for index, valor in enumerate(valores_a_buscar_lista):
    if len(valor) == 11:
        consulta_sql_ccvu = f"SELECT CVU_CVU FROM ccvu..ccvus WHERE CVU_CUIT = '{valor}'"
        
        cursor_ccvu.execute(consulta_sql_ccvu)
        
        filas = cursor_ccvu.fetchall()
        if filas:
            for fila in filas:
                nueva_fila = df_cuentas.iloc[index].copy()
                nueva_fila['cuentas_reportar'] = fila[0]  
                
                nuevas_filas.append(nueva_fila)
        else:
            nuevas_filas.append(df_cuentas.iloc[index])
    else:
        df_cuentas.at[index, 'cuentas_reportar'] = valor
        nuevas_filas.append(df_cuentas.iloc[index])


conexion_ccvu.close()


df_resultado = pd.DataFrame(nuevas_filas)
df_resultado['cuentas_reportar'] = df_resultado['cuentas_reportar'].apply(lambda x: x if len(str(x)) == 22 else '') #validador de cuentas

df_resultado = df_resultado[['id','cuentas_reportar','Bloqueo o desbloqueo']]

IDs_entidades = []
for valor in df_resultado['cuentas_reportar']:
    if valor.startswith('000000'):
        IDs_entidades.append(valor[6:7])
    elif valor.startswith('00000'):
        IDs_entidades.append(valor[5:7])
    elif valor.startswith('0000'):
        IDs_entidades.append(valor[4:7])
    else:
        IDs_entidades.append('0')
df_resultado['IDs_entidades'] = IDs_entidades

IDs_entidades_bancos2 =[]
for valor in df_resultado['cuentas_reportar']:
    if valor.startswith('0'):
        IDs_entidades_bancos2.append(valor[:3])
    else :
        IDs_entidades_bancos2.append('0')

df_resultado['IDs_entidades_bancos']=IDs_entidades_bancos2
df_resultado['IDs_entidades_bancos'] = df_resultado['IDs_entidades_bancos'].str.replace('0','')
df_psps=pd.read_excel('Contactos_PSPS.xlsx',dtype=str)
df_bancos=pd.read_excel('Contactos_Bancos.xlsx',dtype=str)

In [ ]:
import pandas as pd
import pyodbc

df_cuentas = pd.read_excel('registro_tickets.xlsx', dtype=str)
df_psps = pd.read_excel('Contactos_PSPS.xlsx', dtype=str)
df_bancos = pd.read_excel('Contactos_Bancos.xlsx', dtype=str)

valores_a_buscar = df_cuentas.apply(lambda row: row['Cuenta a bloquear'] if pd.notnull(row['Cuenta a bloquear']) else row['CUIT a bloquear'], axis=1)
valores_a_buscar_lista = valores_a_buscar.to_list()

df_cuentas['cuentas_reportar'] = None

conexion_ccvu = pyodbc.connect('Driver={SQL Server}; Server=tcp:10.152.0.42,1433; Database=ccvu; Trusted_connection=yes')
cursor_ccvu = conexion_ccvu.cursor()

nuevas_filas = []


for index, valor in enumerate(valores_a_buscar_lista):
    if len(valor) == 11:
        consulta_sql_ccvu = f"SELECT CVU_CVU FROM ccvu..ccvus WHERE CVU_CUIT = '{valor}'"
        cursor_ccvu.execute(consulta_sql_ccvu)
        filas = cursor_ccvu.fetchall()
        if filas:
            for fila in filas:
                nueva_fila = df_cuentas.iloc[index].copy()
                nueva_fila['cuentas_reportar'] = fila[0]
                nuevas_filas.append(nueva_fila)
        else:
            nuevas_filas.append(df_cuentas.iloc[index])
    else:
        df_cuentas.at[index, 'cuentas_reportar'] = valor
        nuevas_filas.append(df_cuentas.iloc[index])


conexion_ccvu.close()


df_resultado = pd.DataFrame(nuevas_filas)
df_resultado['cuentas_reportar'] = df_resultado['cuentas_reportar'].apply(lambda x: x if len(str(x)) == 22 else '')  # Validador de cuentas


df_resultado = df_resultado[['id', 'cuentas_reportar', 'Bloqueo o desbloqueo']]

IDs_entidades = []
for valor in df_resultado['cuentas_reportar']:
    if valor.startswith('000000'):
        IDs_entidades.append(valor[6:7])
    elif valor.startswith('00000'):
        IDs_entidades.append(valor[5:7])
    elif valor.startswith('0000'):
        IDs_entidades.append(valor[4:7])
    else:
        IDs_entidades.append('0')
df_resultado['IDs_entidades'] = IDs_entidades


IDs_entidades_bancos2 = []
for valor in df_resultado['cuentas_reportar']:
    if valor.startswith('0'):
        IDs_entidades_bancos2.append(valor[:3])
    else:
        IDs_entidades_bancos2.append('0')
df_resultado['IDs_entidades_bancos'] = IDs_entidades_bancos2
df_resultado['IDs_entidades_bancos'] = df_resultado['IDs_entidades_bancos'].str.replace('0', '')

#para obtener el contacto o email
def obtener_contacto(row, df_psps, df_bancos):
    if row['IDs_entidades'] != '0':
        contacto = df_psps[df_psps['ID PSP'] == row['IDs_entidades']]['CONTACTO']
        return contacto.values[0] if not contacto.empty else None
    else:
        email = df_bancos[df_bancos['Código'] == row['IDs_entidades_bancos']]['email']
        return email.values[0] if not email.empty else None

df_resultado['contacto_email'] = df_resultado.apply(obtener_contacto, axis=1, args=(df_psps, df_bancos))



In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas as pd


from_address = 'prueba@mail'
subject = 'prueba'
smtp_server_ip = 'smtp.server.ip'
smtp_port = 587


def enviar_correo(to_address, cuenta_reportar):
    msg = MIMEMultipart()
    msg['From'] = from_address
    msg['To'] = to_address
    msg['Subject'] = subject

    body = '''<html>
    <body style="font-family: Saira, sans-serif;">
    <p>Estimados, buenos días, espero se encuentren bien.</p>
    <p>Hemos detectado un comportamiento poco habitual en unas cuentas pertenecientes a vuestra entidad, dejamos el detalle de dichas transacciones:</p>
    <p><strong>Cuenta con pedido de bloqueo:</strong> {cuenta_reportar}</p>
    <p>Ante cualquier inquietud, no duden en consultarnos.</p>
    <p>Saludos,</p>
    <p style="font-size: smaller;">Por favor, no responda a este correo.</p>
    </body>
    </html>'''.format(cuenta_reportar=cuenta_reportar)

    msg.attach(MIMEText(body, 'html'))

    try:
        server = smtplib.SMTP(smtp_server_ip, smtp_port)
        server.starttls()
        text = msg.as_string()
        server.sendmail(from_address, to_address, text)
        print(f"Correo enviado exitosamente a {to_address}.")
    except Exception as e:
        print(f"Error al enviar el correo a {to_address}: {e}")
    finally:
        server.quit()

for index, row in df_resultado.iterrows():
    to_address = row['contacto_email']
    cuenta_reportar = row['cuentas_reportar']
    if pd.notnull(to_address) and to_address != '':
        enviar_correo(to_address, cuenta_reportar)